In [1]:
#-*-coding:utf-8-*-
import tensorflow as tf
import cv2
import numpy as np
import os
# def read_and_decode(filename):
#     filename_queue = tf.train.string_input_producer([filename])
 
#     reader = tf.TFRecordReader()
#     _, serialized_example = reader.read(filename_queue)
#     features = tf.parse_single_example(serialized_example,
#                                        features={
#                                            'label': tf.FixedLenFeature([], tf.int64),
#                                            'img_raw' : tf.FixedLenFeature([], tf.string),
#                                        })
 
#     img = tf.decode_raw(features['img_raw'], tf.uint8)
#     img = tf.cast(img,tf.float32)
#     img = tf.reshape(img, [224,224,3])*(1./255)-0.5
#     label = tf.cast(features['label'], tf.int32) 
#     return img, label
#print(read_and_decode("Bird_train.tfrecords"))
def conv2d(x, W, b, strides=1):  
        # Conv2D wrapper, with bias and relu activation  
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')  
    x = tf.nn.bias_add(x, b)  
    return tf.nn.relu(x)  
def maxpool2d(x, k=2):  
        # MaxPool2D wrapper  
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')  
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [2]:
import random
dirpath = 'birds'
writer = tf.python_io.TFRecordWriter('Bird_train.tfrecords')
index = 0
classis={}
imgs=[]
labels=[]
for filename in os.listdir(dirpath):
    filepath = os.path.join(dirpath,filename)
    classis[index]=filename
    for img_name in os.listdir(filepath):
        img_path = os.path.join(filepath,img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224,224),interpolation=cv2.INTER_CUBIC)
        imgs.append(img)
        labels.append(index)
    index+=1
print(classis)
def getbetch(num):
    temp1 = []
    temp2=[]
    index=np.random.randint(len(labels),size=num).tolist()
    for i in index:
        temp = np.zeros(6).tolist()
        temp[labels[i]]=1
        temp1.append(imgs[i])
        temp2.append(temp)
    temp1=np.array(temp1).astype(np.float32)
    temp2=np.array(temp2).astype(np.float32)
    return temp1,temp2
print(getbetch(10))
print('ok')

{0: 'egret', 1: 'mandarin', 2: 'owl', 3: 'puffin', 4: 'toucan', 5: 'wood_duck'}
(array([[[[  26.,    9.,   86.],
         [  25.,    6.,   80.],
         [  23.,    3.,   71.],
         ..., 
         [ 170.,  152.,  115.],
         [ 172.,  153.,  116.],
         [ 174.,  156.,  119.]],

        [[  21.,    5.,   83.],
         [  20.,    4.,   79.],
         [  17.,    1.,   70.],
         ..., 
         [ 172.,  154.,  117.],
         [ 172.,  154.,  117.],
         [ 171.,  154.,  117.]],

        [[  20.,    9.,   86.],
         [  21.,    9.,   83.],
         [  20.,    7.,   77.],
         ..., 
         [ 200.,  184.,  147.],
         [ 204.,  187.,  150.],
         [ 206.,  190.,  152.]],

        ..., 
        [[ 138.,  148.,  119.],
         [ 155.,  164.,  134.],
         [ 154.,  162.,  133.],
         ..., 
         [  56.,   42.,   11.],
         [  59.,   44.,   13.],
         [  60.,   46.,   15.]],

        [[ 130.,  138.,  115.],
         [ 135.,  143.,  120.],
     

In [ ]:
x = tf.placeholder(tf.float32, [None, 224,224,3])
y_ = tf.placeholder(tf.float32,[None, 6])#
x=tf.reshape(x, shape=[-1,224,224,3])
conv1_1 = conv2d(x, weight_variable([3,3,3,64]),bias_variable([64]),1)
conv1_2 = conv2d(conv1_1,weight_variable([3,3,64,64]),bias_variable([64]),1)
pool1 = maxpool2d(conv1_2, k = 2)

conv2_1 = conv2d(pool1, weight_variable([3,3,64,128]),bias_variable([128]),1)
conv2_2 = conv2d(conv2_1,weight_variable([3,3,128,128]),bias_variable([128]),1)
pool2 = maxpool2d(conv2_2, k = 2)

conv3_1 = conv2d(pool2, weight_variable([3,3,128,256]),bias_variable([256]),1)
conv3_2 = conv2d(conv3_1,weight_variable([3,3,256,256]),bias_variable([256]),1)
conv3_3 = conv2d(conv3_2,weight_variable([3,3,256,256]),bias_variable([256]),1)
pool3 = maxpool2d(conv3_3, k = 2)

conv4_1 = conv2d(pool3, weight_variable([3,3,256,512]),bias_variable([512]),1)
conv4_2 = conv2d(conv4_1,weight_variable([3,3,512,512]),bias_variable([512]),1)
conv4_3 = conv2d(conv4_2,weight_variable([3,3,512,512]),bias_variable([512]),1)
pool4 = maxpool2d(conv4_3, k = 2)

conv5_1 = conv2d(pool4, weight_variable([3,3,512,512]),bias_variable([512]),1)
conv5_2 = conv2d(conv5_1,weight_variable([3,3,512,512]),bias_variable([512]),1)
conv5_3 = conv2d(conv5_2,weight_variable([3,3,512,512]),bias_variable([512]),1)
pool5 = maxpool2d(conv5_3, k = 2)

h_pool2_flat = tf.reshape(pool5, [-1, 7*7*512])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, weight_variable([7 * 7 * 512, 4096]) + bias_variable([4096])))

#防止过拟合的dropout，keep_prob为起作用的神经元概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, weight_variable([4096, 4096]) + bias_variable([4096])))
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)
#全连接的输出层
#计算输出结果
y_conv = tf.matmul(h_fc2_drop, weight_variable([4096, 6])) +bias_variable([6])

In [ ]:
#交叉熵用于损失函数，指定学习速率
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#tf.argmax可以给出tensor对象在某一维度上数据的最大值所在位置
#tf.equal可以检测预测值是否和真实值匹配，返回一个bool数组，eg:[True, False, True, True]
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
#将bool数组变为[1,0,1,1]，计算出平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        #10个一组，迭代1000次
        haha_x, haha_y = getbetch(50)
     #   print(haha_x.shape,haha_y.shape)
        print('step %d start'%i)
        sess.run(train_step, feed_dict={x: haha_x, y_:haha_y,keep_prob: 0.9})
        saver.save(sess,'Model/model.ckpt')
        ha_x, ha_y = getbetch(50)
        print('step %d'%i,sess.run(accuracy,feed_dict={x:ha_x,y_:ha_y,keep_prob: 1.0}))

step 0 start
step 0 0.14
step 1 start
step 1 0.18
step 2 start
